In [8]:
import os
from binance.client import Client
from datetime import datetime, timedelta
import time
from itertools import count
from itertools import count
import json

API_LIVE = "y49HDNWtWZ2YdQ3lmYrnBfDwv4eTShZEIrgHljYE2MPPmazEiaaw7KzFmS69xJLl"
SECRET_LIVE = "kXCDDLPdUbI3YeLbrgqWqmWR4QebYhtkQL9UydBIVd5CrOO8CB71KDtH7tEf9njv"

SPOT_TEST = "InTPDoHI8blAlKARL9KygcMujUod5KFnZGZhxKz596OG2Q193h2txTA1vOablbDk"
SPOT_SECRET = "7rroq47B9IO0mm31RtlPwhLQ8tDAJijKHL5YBMtrQpkfVls5J74IUA4ykY7jdBto"

In [9]:
TESTNET = True

# authenticate
if TESTNET:
    client = Client(SPOT_TEST, SPOT_SECRET)
    client.API_URL = "https://testnet.binance.vision/api"
else:
    client = Client(API_LIVE, SECRET_LIVE)

In [10]:
"""
VARIABLES
"""

# select what to pair the coins with
PAIR_WITH = 'USDT'
# define size of each trade
QUANTITY = 100
# list of pairs to exclude
FIATS = ['EURUSDT', 'GBPUSDT', 'JPYUSDT', 'USDUSDT', 'DOWN', 'UP']

# the amount of time in MINUTES to calculate the differnce from the current price
TIME_DIFFERENCE = 5
# the difference in % between the first and second checks for the price, by default set at 10 minutes apart.
CHANGE_IN_PRICE = 3
# define in % when to sell a coin that's not making a profit
STOP_LOSS = 3
# define in % when to take profit on a profitable coin
TAKE_PROFIT = 6

In [11]:
"""
COINS TRACKED
"""
coins_bought = {}
coins_bought_file_path = 'coins_bought.json'

# use separate files for testnet and live
if TESTNET:
    coins_bought_file_path = 'testnet_' + coins_bought_file_path

# if saved coins_bought json file exists then load it
if os.path.isfile(coins_bought_file_path):
    with open(coins_bought_file_path) as file:
        coins_bought = json.load(file)

In [15]:
"""
FUNCTIONS
"""

prices = client.get_all_tickers()
prices


[{'symbol': 'BNBBUSD', 'price': '3243.19000000'},
 {'symbol': 'BTCBUSD', 'price': '80928.39000000'},
 {'symbol': 'ETHBUSD', 'price': '77189.42000000'},
 {'symbol': 'LTCBUSD', 'price': '1095.58000000'},
 {'symbol': 'TRXBUSD', 'price': '0.16636000'},
 {'symbol': 'XRPBUSD', 'price': '4.36990000'},
 {'symbol': 'BNBUSDT', 'price': '390.16000000'},
 {'symbol': 'BTCUSDT', 'price': '36200.54000000'},
 {'symbol': 'ETHUSDT', 'price': '2418.49000000'},
 {'symbol': 'LTCUSDT', 'price': '166.17000000'},
 {'symbol': 'TRXUSDT', 'price': '0.07768000'},
 {'symbol': 'XRPUSDT', 'price': '0.78320000'},
 {'symbol': 'BNBBTC', 'price': '0.00986800'},
 {'symbol': 'ETHBTC', 'price': '0.06750200'},
 {'symbol': 'LTCBTC', 'price': '0.01000000'},
 {'symbol': 'TRXBTC', 'price': '0.00000300'},
 {'symbol': 'XRPBTC', 'price': '0.00002720'},
 {'symbol': 'LTCBNB', 'price': '0.43860000'},
 {'symbol': 'TRXBNB', 'price': '0.00020540'},
 {'symbol': 'XRPBNB', 'price': '0.00253100'}]

In [14]:
initial_price = {}

for coin in prices:
    if PAIR_WITH in coin['symbol'] and all(item not in coin['symbol'] for item in FIATS):
            initial_price[coin['symbol']] = { 'price': coin['price'], 'time': datetime.now()}

initial_price

{'BNBUSDT': {'price': '390.30000000',
  'time': datetime.datetime(2021, 6, 5, 23, 15, 17, 159952)},
 'BTCUSDT': {'price': '39317.60000000',
  'time': datetime.datetime(2021, 6, 5, 23, 15, 17, 159961)},
 'ETHUSDT': {'price': '2418.49000000',
  'time': datetime.datetime(2021, 6, 5, 23, 15, 17, 159963)},
 'LTCUSDT': {'price': '166.17000000',
  'time': datetime.datetime(2021, 6, 5, 23, 15, 17, 159966)},
 'TRXUSDT': {'price': '0.07768000',
  'time': datetime.datetime(2021, 6, 5, 23, 15, 17, 159968)},
 'XRPUSDT': {'price': '0.78320000',
  'time': datetime.datetime(2021, 6, 5, 23, 15, 17, 159971)}}

In [ ]:
def get_price():
    initial_price = {}
    prices = client.get_all_tickers()

    for coin in prices:
        # return USDT pairs, exclude margin symbols
        if PAIR_WITH in coin['symbol'] and all(item not in coin['symbol'] for item in FIATS):
            initial_price[coin['symbol']] = { 'price': coin['price'], 'time': datetime.now()}

